In [1]:
import numpy as np
import pandas as pd
from nn_utils import *
from sklearn.preprocessing import OneHotEncoder


In [2]:
net = NeuralNetwork()

net.add_input_layer(7)

net.add_hidden_layer(7, 3)

net.add_output_layer(3, 2)

print(net)

INPUT LAYER: 
NODE 0 OUTPUT = 0.0
NODE 1 OUTPUT = 0.0
NODE 2 OUTPUT = 0.0
NODE 3 OUTPUT = 0.0
NODE 4 OUTPUT = 0.0
NODE 5 OUTPUT = 0.0
NODE 6 OUTPUT = 0.0

LAYER 0 
NODE 0 WEIGHTS = 0.023854438360415998, -0.2910841315619781, -0.18913953348501386, 0.5272374232418281, -0.17711070960464695, -0.30770117330288443, 0.4987386160736377,  BIAS = 0.0
NODE 1 WEIGHTS = -0.6582090561697531, 0.6116964931417952, -0.294602478358846, -0.35371960811222847, -0.4515468781795281, 0.8053392608269576, -0.7499100957178207,  BIAS = 0.0
NODE 2 WEIGHTS = 0.4164202920293154, 0.051967230516604436, 0.3432073313352668, 0.15234273472905072, 0.047781406765268354, 0.20050797634367246, 0.45865781745596207,  BIAS = 0.0


OUTPUT LAYER: 
NODE 0 WEIGHTS = -0.29193860836255003, 0.5263656989520661, 0.549341384209587,  BIAS = 0.0
NODE 1 WEIGHTS = -1.058809676496938, -0.11239263658708766, 0.8057346798333422,  BIAS = 0.0
 


In [6]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]

data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
data.head()

data.reset_index()
tr_data_Y = data[["Class"]]
id_dummies = pd.get_dummies(data["ID"], dtype=int, drop_first = True)

tr_data_X = pd.concat([data.drop(["Class"], axis = 1), id_dummies], axis = 1)

print(tr_data_X)


In [ ]:
losses = net.train(tr_data_X, tr_data_Y, epochs=100,
                 eta=0.01, clip_value=10000, act_fun="relu")
#print(net)
#print(net.input_layer.weights)

plot_loss(losses)


ValueError: shapes (131,) and (7,3) not aligned: 131 (dim 0) != 7 (dim 0)